In [1]:
#V1版本的不足之处;后续添加个检查单原子负载前后基底变化，从而智能化选择更加合适的吸附位
import numpy as np
import pandas as pd
import random
import os
from shutil import copy
from math import cos, sin, atan2, sqrt, pi ,radians, degrees

In [2]:
#在shell里面的SMAZ目录下使用两条命令生成reached_required_accuracy.dat和total_energy.dat
#for i in *;do echo $i fcc-$(grep "reached required" $i/fcc/OUTCAR|cut -b 2-8) hcp-$(grep "reached required" $i/hcp/OUTCAR|cut -b 2-8) top-$(grep "reached required" $i/top/OUTCAR|cut -b 2-8) >> ../reached_required_accuracy.dat;done 
#for i in *;do echo $i fcc-$(grep "energy  without entropy=" $i/fcc/OUTCAR|tail -1|awk '{print $7}') hcp-$(grep "energy  without entropy=" $i/hcp/OUTCAR|tail -1|awk '{print $7}') top-$(grep "energy  without entropy=" $i/top/OUTCAR|tail -1|awk '{print $7}') >> ../total_energy.dat;done 

In [3]:
#把SMAZ目录复制到别处，保留CONTCAR和OUTCAR留作下步脚本处理
#参考命令： 
#  mkdir ../100select_redu && cp -r * ../100select_redu && cd ../100select_redu
#  find . |grep -vE 'CONTCAR'|xargs rm -f

In [4]:
# 打开新文件
def open_poscar(name):
    f = open(name,'r+')
    flist= f.readlines()
    f.close()
    return flist

In [5]:
# 组成新文件数据
def get_Hads_poscar_data(flist, start_num, sample_type,  SMAZ_name):
    if sample_type == 'SMAZ':
        support_name = SMAZ_name.split('_')[-1]
        line1 = [str(x) for x in flist[0].split()]
        line2 = [str(x) for x in flist[1].split()]
        line3 = [str(x) for x in flist[2].split()]
        line4 = [str(x) for x in flist[3].split()]
        line5 = [str(x) for x in flist[4].split()]
        line6 = [str(x) for x in flist[5].split()] + ['H'] # 原子序号
        line7 = [str(x) for x in flist[6].split()] + ['1'] #原子个数
        line8 = [str(x) for x in flist[7].split()]
        num_atoms = sum([int(x) for x in flist[6].split()])
        if  str(line8).strip().upper()[2] == 'D':
            x_cartesian = []
            y_cartesian = []
            z_cartesian = []
            for i in range(start_num, num_atoms + start_num): 
    #         x_cartesian.append(float(line[i].split()[0]) * a1 + float(line[i].split()[1]) * a2 + float(line[i].split()[2]) * a3)
    #         y_cartesian.append(float(line[i].split()[0]) * b1 + float(line[i].split()[1]) * b2 + float(line[i].split()[2]) * b3)
    #         z_cartesian.append(float(line[i].split()[0]) * z1 + float(line[i].split()[1]) * z2 + float(line[i].split()[2]) * z3)
                x_cartesian.append(float(flist[i].split()[0]))
                y_cartesian.append(float(flist[i].split()[1]))
                z_cartesian.append(float(flist[i].split()[2]))
        cartesian_xyz_bare = list(zip(x_cartesian, y_cartesian, z_cartesian))
        atom_name_list = [str(x) for x in flist[5].split()]
        atom_num_list = [str(x) for x in flist[6].split()]
        c = [float(x) for x in flist[4].split()][-1]
        cartesian_xyz_H = get_cartesian_xyz(cartesian_xyz_bare, c, support_name, atom_name_list, atom_num_list)
        cartesian_xyz = cartesian_xyz_bare + cartesian_xyz_H
        new_list = []
        new_list.append(line1)
        new_list.append(line2)
        new_list.append(line3)
        new_list.append(line4)
        new_list.append(line5)
        new_list.append(line6)
        new_list.append(line7)
        new_list.append(line8)
        for i in range(len(cartesian_xyz)):
            new_list.append(cartesian_xyz[i])
    return new_list

In [6]:
def get_cartesian_xyz(cartesian_xyz_bare, c, support_name, atom_name_list, atom_num_list):
    atom_name_list = [str(x) for x in flist[5].split()]
    atom_num_list = [int(x) for x in flist[6].split()]
    MAZ_name = atom_name_list[:-1]
    if support_name in MAZ_name:
        support_num_index = sum(atom_num_list) -1
    else:
        support_index = atom_name_list.index(support_name)
        count = 0
        support_num_index = 0
        for i in range(len(atom_name_list)):
            if count < support_index:
                support_num_index += atom_num_list[i]
                count +=1
    distance_SH = get_radii_sum(support_name, 'H')
    cartesian_xyz_H = [(cartesian_xyz_bare[support_num_index][0], 
                        cartesian_xyz_bare[support_num_index][1],
                        cartesian_xyz_bare[support_num_index][2]+distance_SH/c)]
    return cartesian_xyz_H

In [7]:
# 获取原子半径和
def get_radii_sum(atom1, atom2):
    # 获取涉及到的原子半径
    data = pd.read_excel('共价半径.xlsx', header=None)
    # 生成原子半径字典
    data_radius = dict_radius_production(data)
    radii_sum = data_radius[atom1] + data_radius[atom2]
    return radii_sum

In [8]:
# 生成共价半径字典
def dict_radius_production(data):
    dict_radius = {}
    for i in range(len(data)):
        dict_radius.update({data.iloc[i,0]:data.iloc[i,1]})
    return dict_radius

In [9]:
# 写入新文件
def write_newposcar(path, element_name, new_poscar):
    new_poscar_name = path + '/POSCAR_' + ''.join(element_name)
    f2=open(new_poscar_name,"w")
    for i in range(len(new_poscar)):
        s1 = str(new_poscar[i]).replace('[','').replace(']','').replace('(','').replace(')','')
        s2 = s1.replace("'",'').replace(',','') + '\n' 
        f2.write(s2)
    f2.close()

In [10]:
#读取SMAZ的能量数据
data_energy = pd.read_table('total_energy.dat', sep=' ', header = None)
data_accuracy = pd.read_table('reached_required_accuracy.dat', sep=' ', header = None)
data_energy_new = data_energy.copy()

In [11]:
# 没收敛的能量被替换为0
for i in range(len(data_accuracy)):
    if data_accuracy.iloc[i,1][4:] != 'reached':
        data_energy_new.iloc[i,1] = data_energy_new.iloc[i,1][0:4] + '0'
    if data_accuracy.iloc[i,2][4:] != 'reached':
        data_energy_new.iloc[i,2] = data_energy_new.iloc[i,2][0:4] + '0'
    if data_accuracy.iloc[i,3][4:] != 'reached':
        data_energy_new.iloc[i,3] = data_energy_new.iloc[i,3][0:4] + '0'

In [12]:
# 获取最稳定单原子金属负载点 
data_ads = pd.DataFrame()
data_ads['Sample'] = data_energy.iloc[:,0]
list_ads = []
for i in range(len(data_accuracy)):
    list_row =  [float(data_energy_new.iloc[i,1][4:]), 
            float(data_energy_new.iloc[i,2][4:]), float(data_energy_new.iloc[i,3][4:])]
    min_index = list_row.index(min(list_row))
    list_ads.append(data_energy_new.iloc[i,min_index+1][0:3])
data_ads['ads_site'] = list_ads 

In [13]:
# 生成SMAZ的HER结构
Num_select = 100  #选择个数
name = '20220822' #命名，以时间和英文为命名方式
name_total = name + '_' + str(Num_select)
os.makedirs(name_total+'_HER')
for i in os.listdir(name_total+'_redu'):
    os.makedirs(name_total+'_HER/'+i)
    ads_site = data_ads[data_ads['Sample'].isin([i])]['ads_site'].values
    from_path = name_total+ '_redu/' + i + '/' + ads_site + '/CONTCAR'
    from_path2 = ''.join(from_path) #array转字符串
    to_path = name_total+'_HER/'+ i + '/CONTCAR_' + ads_site
    to_path2 = ''.join(to_path) #array转字符串
    copy(from_path2, to_path2)
#     to_path_II = name+'_HER/'+ '/CONTCAR_' + i + '_' + ads_site
#     to_path_II2 = ''.join(to_path_II)
#     copy(from_path2, to_path_II2)

FileNotFoundError: [WinError 3] 系统找不到指定的路径。: '20220822_100_redu'

In [177]:
# 100select中Zr_C_N_W Zr_C_N_Co Pt_Si_As_Mn Mn_Sn_N_Hf Ir_Sn_As_W Ir_C_N_Y Fe_C_P_Fe这些结构有问题

In [288]:
start_num = 9
for i in os.listdir(name_total+'_HER'):
    path = name_total+'_HER/'+ i
    ads_site = data_ads[data_ads['Sample'].isin([i])]['ads_site'].values
    ads_site2 = ''.join(ads_site)
    flist = open_poscar(name_total+'_HER/'+ i + '/CONTCAR_' + ads_site2)
    supporting_name = i.split('_')[-1]
    new_list = get_Hads_poscar_data(flist, start_num -1 , 'SMAZ', i)
    write_newposcar(path, i, new_list) 